In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary

In [3]:
from tensorflow.keras.utils import to_categorical

In [4]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [5]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

#cat_vars = ['Store', 'DayOfWeek']

In [6]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']
#contin_vars = ['BeforeStateHoliday_bool', 'Max_TemperatureC']

In [7]:
uniques = DataFrameSummary(df[cat_vars]).summary().loc[['uniques']]

In [8]:
uniques.T

,uniques
Store,1115
DayOfWeek,7
Year,3
Month,12
Day,31
StateHoliday,4
CompetitionMonthsOpen,25
Promo2Weeks,26
StoreType,4
Assortment,3


# Asignación de dimensión de embeddings

In [9]:
cat_var_dict = {'Store': 50, 'DayOfWeek': 2, 'Year': 2, 'Month': 2,
'Day': 10, 'StateHoliday': 2, 'CompetitionMonthsOpen': 2,
'Promo2Weeks': 1, 'StoreType': 2, 'Assortment': 3, 'PromoInterval': 3,
'CompetitionOpenSinceYear': 4, 'Promo2SinceYear': 4, 'State': 6,
'Week': 25, 'Events': 4, 'Promo_fw': 1,
'Promo_bw': 1, 'StateHoliday_bool_fw': 1,
'StateHoliday_bool_bw': 1, 'SchoolHoliday_fw': 1,
'SchoolHoliday_bw': 1}

In [10]:
for v in cat_vars:
    uniques_ = df[v].unique()
    uniques_.sort()
    print(v, cat_var_dict[v], len(uniques_), uniques_)
    print()

Store 50 1115 [   0    1    2 ... 1112 1113 1114]

DayOfWeek 2 7 [0 1 2 3 4 5 6]

Year 2 3 [0 1 2]

Month 2 12 [ 0  1  2  3  4  5  6  7  8  9 10 11]

Day 10 31 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]

StateHoliday 2 4 [0 1 2 3]

CompetitionMonthsOpen 2 25 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]

Promo2Weeks 1 26 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]

StoreType 2 4 [0 1 2 3]

Assortment 3 3 [0 1 2]

PromoInterval 3 4 [0 1 2 3]

CompetitionOpenSinceYear 4 21 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]

Promo2SinceYear 4 8 [0 1 2 3 4 5 6 7]

State 6 12 [ 0  1  2  3  4  5  6  7  8  9 10 11]

Week 25 52 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51]

Events 4 22 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20

# Definición de modelo

In [11]:
add_customers = True
log_output = False
output_activation = 'linear'

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, BatchNormalization, Activation, LeakyReLU, Dropout
from tensorflow.keras.regularizers import l2

In [13]:
def get_cat_vars_model(cat_vars, uniques, cat_var_dict):
    cat_vars_embed_outs = []
    cat_var_inputs = []
    for cat_var in cat_vars:
        cat_var_in = Input(shape=(1,), name=f"{cat_var}_input")
        cat_var_inputs.append(cat_var_in)
        embed_out = Embedding(uniques[cat_var][0], cat_var_dict[cat_var], name=f'{cat_var}_Embed')(cat_var_in)
        flatten_out = Flatten(name=f"{cat_var}_flat")(embed_out)
        cat_vars_embed_outs.append(flatten_out)
    return cat_var_inputs, cat_vars_embed_outs

def get_cont_vars_input(contin_vars, dense_layer=False):
    cont_vars_inputs = []
    cont_vars_outputs = []
    for cont_var in contin_vars:
        cont_var_in = Input(shape=(1,), name=f"{cont_var}_input")
        cont_vars_inputs.append(cont_var_in)
        if dense_layer:
            cont_var_out = Dense(1, name=f"{cont_var}_input", activation = 'linear')(cont_var_in)
            cont_vars_outputs.append(cont_var_out)
        else:
            cont_vars_outputs.append(cont_var_in)
    return cont_vars_inputs, cont_vars_outputs

In [14]:
cat_var_inputs, cat_vars_embed_outs = get_cat_vars_model(cat_vars, uniques, cat_var_dict)
cont_vars_inputs,  cont_vars_outs= get_cont_vars_input(contin_vars)

In [15]:
# Descomentar para ver resultados
# cat_vars_embed_outs
# cat_var_inputs
# cont_vars_inputs
# cont_vars_outs

In [16]:
first_hidden_units = 1000
second_hidden_units = 500
l2_lambda = 1e-3
merged = Concatenate(name='All_Concatenate')(cat_vars_embed_outs + cont_vars_inputs)
x = Dense(first_hidden_units, kernel_initializer="uniform", kernel_regularizer=l2(l2_lambda))(merged)
# x = BatchNormalization()(x)
x = Activation('relu')(x)
# x = LeakyReLU()(x)
x = Dense(second_hidden_units, kernel_initializer="uniform", kernel_regularizer=l2(l2_lambda))(x)
# x = BatchNormalization()(x)
x = Activation('relu')(x)
# x = LeakyReLU()(x)

output_1 = Dense(1, name='Sales', activation=output_activation)(x)
output_2 = Dense(1, name='Customers', activation=output_activation)(x)

In [17]:
if add_customers:
    model = Model(cat_var_inputs + cont_vars_inputs, [output_1, output_2])
else: 
    model = Model(cat_var_inputs + cont_vars_inputs, [output_1])

In [18]:
# Descomentar para ver
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Store_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
DayOfWeek_input (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
Year_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Month_input (InputLayer)        [(None, 1)]          0                                            
______________________________________________________________________________________________

In [19]:
model.save_weights('initial_weights.hdf5')

In [20]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [21]:
from matplotlib import pyplot as plt
# plt.figure(figsize=(20,5))
# plt.plot((df_train[df_train['Store']==1]['Sales'].values - df_train[df_train['Store']==1]['Sales'].mean())/df_train[df_train['Store']==1]['Sales'].std())
# plt.show()
# plt.figure(figsize=(20,5))
# plt.plot(np.log(df_train[df_train['Store']==1]['Sales'].values)/np.max(np.log(df_train[df_train['Store']==1]['Sales'].values)))
# plt.show()

In [22]:
stores_mean = {}
for store, g_df in df_train.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [23]:
df_train.loc[:, 'mean_by_store'] = df_train['Store'].apply(stores_mean.get)
df_val.loc[:, 'mean_by_store'] = df_val['Store'].apply(stores_mean.get)

df_train.loc[:, 'Sales_store'] = df_train['Sales'] - df_train['mean_by_store']
df_val.loc[:, 'Sales_store'] = df_val['Sales'] - df_val['mean_by_store']

C:\Users\saraj\anaconda3\envs\DL\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\saraj\anaconda3\envs\DL\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [24]:
from tensorflow.keras import backend as K

In [25]:
def rmspe(y_true, y_pred):
    return K.sqrt(K.mean(K.square((y_true - y_pred)/y_true)))

In [26]:
def get_metric(df, sales_):
    return np.sqrt((((df['Sales'] - sales_)/df['Sales'])**2).mean())

In [27]:
get_metric(df_val, df_val['mean_by_store'])

0.3071746085338871

In [28]:
all_vars = cat_vars + contin_vars
X_train = np.hsplit(df_train[all_vars].values, len(all_vars))
X_val = np.hsplit(df_val[all_vars].values, len(all_vars))
X_test = np.hsplit(df_test[all_vars].values, len(all_vars))

In [29]:
if add_customers:
    y_out_columns = ['Sales', 'Customers']
else:
    y_out_columns = ['Sales_store']
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    #y_mean = df_train[y_out_columns].mean().values
    #y_std = df_train[y_out_columns].std().values
    #y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_max = df_train[y_out_columns].max().values
    y_train = df_train[y_out_columns].values/y_max
    y_val = df_val[y_out_columns].values/y_max
y_train = np.hsplit(y_train, y_train.shape[1])
y_val = np.hsplit(y_val, y_val.shape[1])

In [30]:
lr = 0.001
model.compile(optimizer=Adam(lr=lr), metrics=['mse', rmspe], loss='mse')

In [31]:
if add_customers:
    checkpoint = ModelCheckpoint('bestmodel.hdf5', monitor='val_Sales_mse', verbose=1, save_best_only=True)
else:
    checkpoint = ModelCheckpoint('bestmodel.hdf5', monitor='val_loss', verbose=1, save_best_only=True)

In [32]:
epochs = 500
batch_size = 128
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),  epochs=epochs, batch_size=batch_size, callbacks=[checkpoint], verbose=2)

Epoch 1/500
6361/6361 - 65s - loss: 0.0067 - Sales_loss: 0.0012 - Customers_loss: 4.8821e-04 - Sales_mse: 0.0012 - Sales_rmspe: 0.2572 - Customers_mse: 4.8821e-04 - Customers_rmspe: 0.2557 - val_loss: 0.0013 - val_Sales_loss: 8.2367e-04 - val_Customers_loss: 3.0905e-04 - val_Sales_mse: 8.2367e-04 - val_Sales_rmspe: 0.1871 - val_Customers_mse: 3.0905e-04 - val_Customers_rmspe: 0.1692

Epoch 00001: val_Sales_mse improved from inf to 0.00082, saving model to bestmodel.hdf5
Epoch 2/500
6361/6361 - 58s - loss: 0.0014 - Sales_loss: 8.8997e-04 - Customers_loss: 3.2042e-04 - Sales_mse: 8.8997e-04 - Sales_rmspe: 0.2344 - Customers_mse: 3.2042e-04 - Customers_rmspe: 0.2248 - val_loss: 0.0012 - val_Sales_loss: 7.6849e-04 - val_Customers_loss: 2.5646e-04 - val_Sales_mse: 7.6849e-04 - val_Sales_rmspe: 0.1759 - val_Customers_mse: 2.5646e-04 - val_Customers_rmspe: 0.1692

Epoch 00002: val_Sales_mse improved from 0.00082 to 0.00077, saving model to bestmodel.hdf5
Epoch 3/500
6361/6361 - 58s - loss: 0.


Epoch 00018: val_Sales_mse did not improve from 0.00065
Epoch 19/500
6361/6361 - 58s - loss: 9.6850e-04 - Sales_loss: 6.4819e-04 - Customers_loss: 2.2932e-04 - Sales_mse: 6.4819e-04 - Sales_rmspe: 0.1817 - Customers_mse: 2.2932e-04 - Customers_rmspe: 0.1595 - val_loss: 0.0010 - val_Sales_loss: 6.8944e-04 - val_Customers_loss: 2.2648e-04 - val_Sales_mse: 6.8944e-04 - val_Sales_rmspe: 0.1603 - val_Customers_mse: 2.2648e-04 - val_Customers_rmspe: 0.1398

Epoch 00019: val_Sales_mse did not improve from 0.00065
Epoch 20/500
6361/6361 - 58s - loss: 9.6714e-04 - Sales_loss: 6.4564e-04 - Customers_loss: 2.2953e-04 - Sales_mse: 6.4564e-04 - Sales_rmspe: 0.1808 - Customers_mse: 2.2953e-04 - Customers_rmspe: 0.1592 - val_loss: 0.0010 - val_Sales_loss: 6.8350e-04 - val_Customers_loss: 2.4954e-04 - val_Sales_mse: 6.8350e-04 - val_Sales_rmspe: 0.1613 - val_Customers_mse: 2.4954e-04 - val_Customers_rmspe: 0.1594

Epoch 00020: val_Sales_mse did not improve from 0.00065
Epoch 21/500
6361/6361 - 58s - 


Epoch 00036: val_Sales_mse did not improve from 0.00065
Epoch 37/500
6361/6361 - 57s - loss: 9.5133e-04 - Sales_loss: 6.4163e-04 - Customers_loss: 2.2576e-04 - Sales_mse: 6.4163e-04 - Sales_rmspe: 0.1801 - Customers_mse: 2.2576e-04 - Customers_rmspe: 0.1583 - val_loss: 0.0010 - val_Sales_loss: 7.0722e-04 - val_Customers_loss: 2.3301e-04 - val_Sales_mse: 7.0722e-04 - val_Sales_rmspe: 0.1525 - val_Customers_mse: 2.3301e-04 - val_Customers_rmspe: 0.1322

Epoch 00037: val_Sales_mse did not improve from 0.00065
Epoch 38/500
6361/6361 - 58s - loss: 9.5039e-04 - Sales_loss: 6.4124e-04 - Customers_loss: 2.2567e-04 - Sales_mse: 6.4124e-04 - Sales_rmspe: 0.1805 - Customers_mse: 2.2567e-04 - Customers_rmspe: 0.1588 - val_loss: 0.0010 - val_Sales_loss: 7.1455e-04 - val_Customers_loss: 2.3259e-04 - val_Sales_mse: 7.1455e-04 - val_Sales_rmspe: 0.1603 - val_Customers_mse: 2.3259e-04 - val_Customers_rmspe: 0.1436

Epoch 00038: val_Sales_mse did not improve from 0.00065
Epoch 39/500
6361/6361 - 57s - 


Epoch 00054: val_Sales_mse did not improve from 0.00064
Epoch 55/500
6361/6361 - 58s - loss: 9.4389e-04 - Sales_loss: 6.3915e-04 - Customers_loss: 2.2374e-04 - Sales_mse: 6.3915e-04 - Sales_rmspe: 0.1799 - Customers_mse: 2.2374e-04 - Customers_rmspe: 0.1576 - val_loss: 0.0011 - val_Sales_loss: 7.3300e-04 - val_Customers_loss: 2.3995e-04 - val_Sales_mse: 7.3300e-04 - val_Sales_rmspe: 0.1608 - val_Customers_mse: 2.3995e-04 - val_Customers_rmspe: 0.1427

Epoch 00055: val_Sales_mse did not improve from 0.00064
Epoch 56/500
6361/6361 - 58s - loss: 9.4701e-04 - Sales_loss: 6.4104e-04 - Customers_loss: 2.2448e-04 - Sales_mse: 6.4104e-04 - Sales_rmspe: 0.1802 - Customers_mse: 2.2448e-04 - Customers_rmspe: 0.1580 - val_loss: 0.0010 - val_Sales_loss: 7.1397e-04 - val_Customers_loss: 2.4095e-04 - val_Sales_mse: 7.1397e-04 - val_Sales_rmspe: 0.1638 - val_Customers_mse: 2.4095e-04 - val_Customers_rmspe: 0.1432

Epoch 00056: val_Sales_mse did not improve from 0.00064
Epoch 57/500
6361/6361 - 58s - 


Epoch 00072: val_Sales_mse did not improve from 0.00064
Epoch 73/500
6361/6361 - 58s - loss: 9.3951e-04 - Sales_loss: 6.3789e-04 - Customers_loss: 2.2172e-04 - Sales_mse: 6.3789e-04 - Sales_rmspe: 0.1798 - Customers_mse: 2.2172e-04 - Customers_rmspe: 0.1578 - val_loss: 0.0010 - val_Sales_loss: 7.2376e-04 - val_Customers_loss: 2.4122e-04 - val_Sales_mse: 7.2376e-04 - val_Sales_rmspe: 0.1596 - val_Customers_mse: 2.4122e-04 - val_Customers_rmspe: 0.1345

Epoch 00073: val_Sales_mse did not improve from 0.00064
Epoch 74/500
6361/6361 - 58s - loss: 9.3969e-04 - Sales_loss: 6.3822e-04 - Customers_loss: 2.2169e-04 - Sales_mse: 6.3822e-04 - Sales_rmspe: 0.1792 - Customers_mse: 2.2169e-04 - Customers_rmspe: 0.1570 - val_loss: 0.0010 - val_Sales_loss: 7.1345e-04 - val_Customers_loss: 2.2770e-04 - val_Sales_mse: 7.1345e-04 - val_Sales_rmspe: 0.1492 - val_Customers_mse: 2.2770e-04 - val_Customers_rmspe: 0.1306

Epoch 00074: val_Sales_mse did not improve from 0.00064
Epoch 75/500
6361/6361 - 57s - 

...


Epoch 00306: val_Sales_mse did not improve from 0.00064
Epoch 307/500
6361/6361 - 58s - loss: 9.1925e-04 - Sales_loss: 6.2718e-04 - Customers_loss: 2.1930e-04 - Sales_mse: 6.2718e-04 - Sales_rmspe: 0.1766 - Customers_mse: 2.1930e-04 - Customers_rmspe: 0.1565 - val_loss: 9.8223e-04 - val_Sales_loss: 6.7482e-04 - val_Customers_loss: 2.3420e-04 - val_Sales_mse: 6.7482e-04 - val_Sales_rmspe: 0.1473 - val_Customers_mse: 2.3420e-04 - val_Customers_rmspe: 0.1323

Epoch 00307: val_Sales_mse did not improve from 0.00064
Epoch 308/500
6361/6361 - 57s - loss: 9.2107e-04 - Sales_loss: 6.2856e-04 - Customers_loss: 2.1919e-04 - Sales_mse: 6.2856e-04 - Sales_rmspe: 0.1768 - Customers_mse: 2.1919e-04 - Customers_rmspe: 0.1568 - val_loss: 0.0011 - val_Sales_loss: 7.4839e-04 - val_Customers_loss: 2.7119e-04 - val_Sales_mse: 7.4839e-04 - val_Sales_rmspe: 0.1604 - val_Customers_mse: 2.7119e-04 - val_Customers_rmspe: 0.1437

Epoch 00308: val_Sales_mse did not improve from 0.00064
Epoch 309/500
6361/6361 -


Epoch 00324: val_Sales_mse did not improve from 0.00064
Epoch 325/500
6361/6361 - 58s - loss: 9.2080e-04 - Sales_loss: 6.2849e-04 - Customers_loss: 2.1932e-04 - Sales_mse: 6.2849e-04 - Sales_rmspe: 0.1766 - Customers_mse: 2.1932e-04 - Customers_rmspe: 0.1566 - val_loss: 0.0011 - val_Sales_loss: 7.8604e-04 - val_Customers_loss: 2.7262e-04 - val_Sales_mse: 7.8604e-04 - val_Sales_rmspe: 0.1739 - val_Customers_mse: 2.7262e-04 - val_Customers_rmspe: 0.1625

Epoch 00325: val_Sales_mse did not improve from 0.00064
Epoch 326/500
6361/6361 - 57s - loss: 9.1947e-04 - Sales_loss: 6.2749e-04 - Customers_loss: 2.1885e-04 - Sales_mse: 6.2749e-04 - Sales_rmspe: 0.1767 - Customers_mse: 2.1885e-04 - Customers_rmspe: 0.1567 - val_loss: 0.0010 - val_Sales_loss: 6.9332e-04 - val_Customers_loss: 2.6384e-04 - val_Sales_mse: 6.9332e-04 - val_Sales_rmspe: 0.1588 - val_Customers_mse: 2.6384e-04 - val_Customers_rmspe: 0.1505

Epoch 00326: val_Sales_mse did not improve from 0.00064
Epoch 327/500
6361/6361 - 57s


Epoch 00342: val_Sales_mse did not improve from 0.00064
Epoch 343/500
6361/6361 - 57s - loss: 9.1801e-04 - Sales_loss: 6.2631e-04 - Customers_loss: 2.1871e-04 - Sales_mse: 6.2631e-04 - Sales_rmspe: 0.1762 - Customers_mse: 2.1871e-04 - Customers_rmspe: 0.1562 - val_loss: 0.0010 - val_Sales_loss: 7.2523e-04 - val_Customers_loss: 2.1902e-04 - val_Sales_mse: 7.2523e-04 - val_Sales_rmspe: 0.1547 - val_Customers_mse: 2.1902e-04 - val_Customers_rmspe: 0.1324

Epoch 00343: val_Sales_mse did not improve from 0.00064
Epoch 344/500
6361/6361 - 58s - loss: 9.2013e-04 - Sales_loss: 6.2816e-04 - Customers_loss: 2.1891e-04 - Sales_mse: 6.2816e-04 - Sales_rmspe: 0.1765 - Customers_mse: 2.1891e-04 - Customers_rmspe: 0.1565 - val_loss: 0.0010 - val_Sales_loss: 7.3310e-04 - val_Customers_loss: 2.4460e-04 - val_Sales_mse: 7.3310e-04 - val_Sales_rmspe: 0.1653 - val_Customers_mse: 2.4460e-04 - val_Customers_rmspe: 0.1422

Epoch 00344: val_Sales_mse did not improve from 0.00064
Epoch 345/500
6361/6361 - 58s


Epoch 00360: val_Sales_mse improved from 0.00064 to 0.00064, saving model to bestmodel.hdf5
Epoch 361/500


KeyboardInterrupt: 

In [33]:
plt.plot(history.history['val_Sales_rmspe'])
# plt.plot(history.history['val_Customers_mse'])
# plt.plot(history.history['val_Sales_mse'])
plt.show()
plt.plot(history.history['loss'])
# plt.plot(history.history['Customers_mse'])
# plt.plot(history.history['Sales_mse'])

NameError: name 'history' is not defined

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [34]:
model.evaluate(X_val, y_val)

944/944 [==============================] - 5s 2ms/step - loss: 9.8692e-04 - Sales_loss: 6.8651e-04 - Customers_loss: 2.2771e-04 - Sales_mse: 6.8651e-04 - Sales_rmspe: 0.1498 - Customers_mse: 2.2771e-04 - Customers_rmspe: 0.1337


[0.0009869240457192063,
 0.0006865132600069046,
 0.00022771133808419108,
 0.0006865132600069046,
 0.14976276457309723,
 0.00022771133808419108,
 0.1337023377418518]

In [35]:
model.load_weights('bestmodel.hdf5')
model.evaluate(X_val, y_val)

944/944 [==============================] - 5s 2ms/step - loss: 9.3501e-04 - Sales_loss: 6.3763e-04 - Customers_loss: 2.2386e-04 - Sales_mse: 6.3763e-04 - Sales_rmspe: 0.1529 - Customers_mse: 2.2386e-04 - Customers_rmspe: 0.1380


[0.000935007119551301,
 0.0006376315141096711,
 0.00022385705960914493,
 0.0006376315141096711,
 0.15292011201381683,
 0.00022385705960914493,
 0.1379983127117157]

In [ ]:
# model.load_weights('bestmodel.hdf5')
# model.evaluate(X_val, y_val)
# 30188/30188 [==============================] - 5s 172us/step
# [0.12197033089921382,
#  0.07211007360268763,
#  0.037183713050426136,
#  0.07211007360268763,
#  0.037183713050426136]

# [0.12932546436786652,
#  0.07751645147800446,
#  0.039259567856788635,
#  0.07751645147800446,
#  0.039259567856788635]

In [36]:
if log_output:
    if add_customers:
        y_pred = np.exp(model.predict(X_val, verbose=1)[0][:, 0]*max_log_y[0])
        y_pred_test = np.exp(model.predict(X_test, verbose=1)[0][:, 0]*max_log_y[0])
    else:
        y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)[:,0]
        y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)[:,0]
else:
    if add_customers:
        #y_pred = (model.predict(X_val, verbose=1)[0]*y_std[0] + y_mean[0])[:,0]
        #y_pred_test = (model.predict(X_test, verbose=1)[0]*y_std[0] + y_mean[0])[:,0]
        y_pred = model.predict(X_val, verbose=1)[0]*y_max
        y_pred_test = model.predict(X_test, verbose=1)[0]*y_max  
    else:
#         y_pred = model.predict(X_val, verbose=1)[:,0]*y_std + y_mean
#         y_pred_test = model.predict(X_test, verbose=1)[:,0]*y_std + y_mean
        y_pred = model.predict(X_val, verbose=1)[:,0]*y_max
        y_pred_test = model.predict(X_test, verbose=1)[:,0]*y_max
y_pred_test[df_test['Open'] == 0] = 0

1284/1284 [==============================] - 7s 3ms/step


In [37]:
np.sqrt((((df_val['Sales'].values - y_pred[:,0])/df_val['Sales'].values)**2).sum()/len(y_pred))

0.16286950698794048

In [ ]:

df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [ ]:
df_test.groupby("Open").count()

In [ ]:
df_test[df_test["Open"!=1]]

# Baseline

In [ ]:
import pandas as pd
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')

In [ ]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [ ]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [ ]:
df_test[['Store', 'Sales']].head(10)

In [ ]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

In [ ]:
sample_csv['Sales'] = df_test['Sales']

In [ ]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [ ]:
sample_csv.head()

# Sumbit a la competición

In [38]:

sample_csv = pd.read_csv('sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_{add_customers}-{log_output}-{output_activation}-{l2_lambda}-{first_hidden_units}-{epochs}-{batch_size}-{lr}.csv', index=False)


# Analisis de embedings

In [ ]:
def plot_embed(layer_name, cat_names):
    Y = model.get_layer(layer_name).get_weights()[0]
    print(Y.shape)
    plt.figure(figsize=(8,8))
    plt.scatter(-Y[:, 0], -Y[:, 1])
    for i, txt in enumerate(cat_names):
        plt.annotate(txt, (-Y[i, 0],-Y[i, 1]), xytext = (-5, 8), textcoords = 'offset points')

In [ ]:
model.load_weights('bestmodel.hdf5')
plot_embed('DayOfWeek_Embed', ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat','Sun'])

In [ ]:
plot_embed('Month_Embed', list(range(12)))

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [ ]:
day_of_week_embedding.shape

In [ ]:
tsne = TSNE(n_components=2, random_state=1, learning_rate=10, n_iter=10000)
Y = tsne.fit_transform(day_of_week_embedding)
names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat','Sun']
plt.figure(figsize=(8,8))
plt.scatter(-Y[:, 0], -Y[:, 1])
for i, txt in enumerate(names):
    plt.annotate(txt, (-Y[i, 0],-Y[i, 1]), xytext = (-5, 8), textcoords = 'offset points')

In [ ]:
cat_vars

## Store embeddings

In [39]:
submodel = Model(cat_var_inputs + cont_vars_inputs, merged)

In [40]:
submodel.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Store_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
DayOfWeek_input (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
Year_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Month_input (InputLayer)        [(None, 1)]          0                                            
____________________________________________________________________________________________

In [41]:
submodel.save('embeddings_model.hdf5')

In [42]:
model.get_weights()

[array([[ 0.08702145, -0.06031736, -0.09607303, ...,  4.2458954 ,
         -0.05407878, -0.01038816],
        [-0.02315388, -0.22001602,  0.29376018, ...,  1.3177052 ,
         -0.12012465,  0.1504277 ],
        [-0.20536374,  0.18841398,  0.1629561 , ..., -0.21185224,
          0.18564324, -0.05293714],
        ...,
        [-0.09395552,  0.05398516,  0.2584791 , ...,  1.713849  ,
         -0.02879323, -0.02986156],
        [ 0.1594073 ,  0.20863743, -0.07959363, ..., -3.0893877 ,
          0.04157081,  0.09151175],
        [ 0.0282048 , -0.24081947, -0.03976118, ...,  1.3615695 ,
         -0.20265664, -0.13634482]], dtype=float32),
 array([[-3.9589977e+00,  2.1169019e+00],
        [-1.2995322e+00, -1.7460362e+00],
        [ 4.1219574e-01, -2.1313231e+00],
        [ 2.3436112e-02, -1.7200034e+00],
        [ 7.7069050e-01,  2.0384583e+00],
        [ 1.9821920e+01,  8.2175627e-02],
        [ 2.5886683e+01,  3.5233655e+00]], dtype=float32),
 array([[-1.6854119e-02,  1.7909966e+00],
     

In [43]:
submodel.get_weights()

[array([[ 0.08702145, -0.06031736, -0.09607303, ...,  4.2458954 ,
         -0.05407878, -0.01038816],
        [-0.02315388, -0.22001602,  0.29376018, ...,  1.3177052 ,
         -0.12012465,  0.1504277 ],
        [-0.20536374,  0.18841398,  0.1629561 , ..., -0.21185224,
          0.18564324, -0.05293714],
        ...,
        [-0.09395552,  0.05398516,  0.2584791 , ...,  1.713849  ,
         -0.02879323, -0.02986156],
        [ 0.1594073 ,  0.20863743, -0.07959363, ..., -3.0893877 ,
          0.04157081,  0.09151175],
        [ 0.0282048 , -0.24081947, -0.03976118, ...,  1.3615695 ,
         -0.20265664, -0.13634482]], dtype=float32),
 array([[-3.9589977e+00,  2.1169019e+00],
        [-1.2995322e+00, -1.7460362e+00],
        [ 4.1219574e-01, -2.1313231e+00],
        [ 2.3436112e-02, -1.7200034e+00],
        [ 7.7069050e-01,  2.0384583e+00],
        [ 1.9821920e+01,  8.2175627e-02],
        [ 2.5886683e+01,  3.5233655e+00]], dtype=float32),
 array([[-1.6854119e-02,  1.7909966e+00],
     